# **Lý Phi Long - 1611146**

In [1]:
#import thư viện cần thiết
## thư viện machine learning và hỗ trợ
import tensorflow as tf
from tensorflow import keras
import numpy as np

## thư viện để vẽ đồ thị
import matplotlib.pyplot as plt

In [0]:
TF_KERAS = 1 

In [0]:
## Import các layer cần thiết
from tensorflow.keras.layers import Input, Dense, DepthwiseConv2D, Convolution2D, MaxPool2D, BatchNormalization, ReLU, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.regularizers import l2

## Định nghĩa 1 Depthwise Separable Convolution
class BottleneckDepthSeparableConvolutionWithResiduals(keras.layers.Layer):

    def __init__(self, n_filters=64, expansion_factor=6, l2_regularizer=0.0, down_sampling=False,strides = [1,1]):
        ## Gọi hàm khởi tạo của keras.layers.Layer và lưu lại các thông số
        super(BottleneckDepthSeparableConvolutionWithResiduals, self).__init__()
        self.n_filters = n_filters
        self.expansion_factor = expansion_factor
        self.down_sampling = down_sampling
        self.l2_regularizer = l2_regularizer
        self.strides = strides
  #     self.shortcut = shortcut
    
    def get_config(self):
        config = super(BottleneckDepthSeparableConvolutionWithResiduals, self).get_config()
        config.update({
            "n_filters": self.n_filters,
            "down_sampling": self.down_sampling,
            "strides": self.strides
        })
        return config
    
    
    
    
    def build(self, input_shape):
      ## Xác định xem input_shape có bằng output_shape không
        self.projection_shortcut = (int(input_shape[3]) == self.n_filters)
        
        
        ## Nếu cần down sampling thì convolutional layer dùng strides=[2,2], kiểu padding là valid
        strides = [1,1]
        pad = 'same'         
        if self.down_sampling:
            strides = [2,2]
            pad = 'valid'
            
          
        channel = input_shape[3]
        
        ##Khai báo các layer 
        
        ##Block 1
        self.expansion_conv = Convolution2D(filters=channel*self.expansion_factor,
                                            kernel_size=[1,1],
                                            strides=[1,1],
                                            padding='same',
                                            use_bias=False,
                                            kernel_regularizer=l2(self.l2_regularizer),
                                            activation=None)
        self.expansion_batch = BatchNormalization()
        self.expansion_relu = ReLU(6.)     
        
        
        ##Block 2
        self.depthwise_conv = DepthwiseConv2D(kernel_size=[3,3],
                                              strides=self.strides,
                                              padding=pad,
                                            use_bias=False,
                                              activation=None)
        self.depthwise_batch = BatchNormalization()
        self.depthwise_relu = ReLU(6.)
        

        ##Block 3
        self.projection_conv = Convolution2D(filters=self.n_filters,
                                            kernel_size=[1,1],
                                            strides=[1,1],
                                            padding='same',
                                            use_bias=False,
                                            kernel_regularizer=l2(self.l2_regularizer),
                                            activation=None)
        self.projection_batch = BatchNormalization()
        
        
        
        
        #self.pointwise_relu = ReLU()
  
    def call(self, inputs):
        
        ## Thiết lập các input cho các layer đã khai báo
        x = inputs
        x = self.expansion_conv(x)
        x = self.expansion_batch(x)
        x = self.expansion_relu(x)
        
        if self.down_sampling == True:
          x = ZeroPadding2D(padding=((0, 1), (0, 1)))(x)
          
        x = self.depthwise_conv(x)
        x = self.depthwise_batch(x)
        x = self.depthwise_relu(x)
        
        x = self.projection_conv(x)
        x = self.projection_batch(x)
        
        if self.projection_shortcut:
            x += inputs
        
        return x


In [28]:
l2_regularizer_rate = 0.001


inputs = keras.layers.Input(shape=(32,32,1))

pad1 = ZeroPadding2D(padding=((0, 1), (0, 1)))(inputs)

conv1 = Convolution2D(filters=32,
                      kernel_size=[3,3],
                      strides=[2,2],
                      padding='valid',
                      use_bias=False,
                      kernel_regularizer=l2(l2_regularizer_rate),
                      activation=None)(pad1)

batch1 = BatchNormalization()(conv1)        

relu1 = ReLU()(batch1)

##Bottleneck1

expansion_conv_1 = DepthwiseConv2D(kernel_size=[3,3],
                strides=[1,1],
                padding='same',
                use_bias=False,
                activation=None)(relu1)

expansion_batch_1 = BatchNormalization()(expansion_conv_1)

expansion_ReLU_1 = ReLU(6.)(expansion_batch_1)

projection_conv_1 = Convolution2D(filters=16,
                      kernel_size=[1,1],
                      strides=[1,1],
                      padding='same',
                      use_bias=False,
                      kernel_regularizer=l2(l2_regularizer_rate),
                      activation=None)(expansion_ReLU_1)
projection_batch_1 = BatchNormalization()(projection_conv_1)



##Bottleneck2
dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=24,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=True)(projection_batch_1)

dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=24,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

##Bottleneck3
dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=32,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=True)(dw_pw_conv)

dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=32,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=32,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

##Bottleneck4
dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=64,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=True)(dw_pw_conv)

dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=64,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=64,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=64,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

##Bottleneck5
dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=96,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=96,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=96,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)
##Bottleneck6
dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=160,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=True)(dw_pw_conv)

dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=160,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=160,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

##Bottleneck7
dw_pw_conv = BottleneckDepthSeparableConvolutionWithResiduals(n_filters=320,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)
#####
conv2 = Convolution2D(filters=1280,
                      kernel_size=[1,1],
                      strides=[1,1],
                      padding='same',
                      use_bias=False,
                      kernel_regularizer=l2(l2_regularizer_rate),
                      activation=None)(dw_pw_conv)
batch2 = BatchNormalization()(conv2)
relu2 = ReLU(6.)(batch2)


avage_pool = GlobalAveragePooling2D()(relu2)

softmax = Dense(units=10, activation='softmax')(avage_pool)

## Compile model
model = keras.models.Model(inputs=inputs, outputs=softmax)
model.compile(optimizer= 'adam' , loss= keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])


## In toàn bộ cấu trúc của model
print("Cấu trúc của model: ")
model.summary()

Cấu trúc của model: 
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
zero_padding2d_9 (ZeroPaddin (None, 33, 33, 1)         0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 16, 16, 32)        288       
_________________________________________________________________
batch_normalization_61 (Batc (None, 16, 16, 32)        128       
_________________________________________________________________
re_lu_41 (ReLU)              (None, 16, 16, 32)        0         
_________________________________________________________________
depthwise_conv2d_21 (Depthwi (None, 16, 16, 32)        288       
_________________________________________________________________
batch_normalization_62 (Batc (None, 16

In [29]:
mobilenetv2=keras.applications.mobilenet_v2.MobileNetV2(input_shape=(32,32,1), 
                                                        alpha =1, 
                                                        include_top=True,
                                                        weights= None, 
                                                        input_tensor=None, 
                                                        pooling=None, 
                                                        classes=10)

mobilenetv2.summary()

Model: "mobilenetv2_1.00_32"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 33, 33, 1)    0           input_7[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 16, 16, 32)   288         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 16, 16, 32)   128         Conv1[0][0]                      
________________________________________________________________________________

In [30]:
## tải MNIST dataset từ keras
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
##resacle ảnh thành ảnh thực trong đoạn [0,1]
X_train, X_test = X_train/255.0, X_test/255.0

##in dataset
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [31]:
import cv2
resized_img = cv2.resize(X_train[0], dsize=(32,32))
print("Kích thước ảnh sau resize: ", resized_img.shape)

Kích thước ảnh sau resize:  (32, 32)


In [32]:
## Resize toàn bộ ảnh train tập train và test
X_train = np.array([cv2.resize(img, dsize=(32,32)) for img in X_train])
X_test = np.array([cv2.resize(img, dsize=(32,32)) for img in X_test])
print("Kích thước tập sau khi resize: ", X_train.shape, X_test.shape)

## Reshape ảnh để phù hợp với input của model (thêm một trục)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
print("Kích thước tập sau khi reshape: ", X_train.shape, X_test.shape)

#Tách một phần tập train thành tập valid
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1)

## Reshape ảnh để phù hợp với input của model (thêm một trục)

Kích thước tập sau khi resize:  (60000, 32, 32) (10000, 32, 32)
Kích thước tập sau khi reshape:  (60000, 32, 32, 1) (10000, 32, 32, 1)


In [33]:
mc = keras.callbacks.ModelCheckpoint(filepath="mobilenetv2.h5", 
                                     monitor='val_loss',
                                     mode='min', 
                                     verbose=0, 
                                     save_best_only=True)

## Train  ## Khuyến cáo chạy COLAB (hoặc tương tự)
history = model.fit(X_train, y_train,
                    batch_size=100,
                    epochs=10,
                    validation_data=(X_valid, y_valid),
                    callbacks=[mc])                    


## Đánh giá model trên tập test
valid_loss, valid_acc = model.evaluate(X_valid, y_valid)
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Valid: loss {} acc {} -- Test: loss {} valid {}".format(valid_loss, valid_acc, test_loss, test_acc))

## Load lại model tốt nhất đã lưu
print("best model: ")
model.load_weights("mobilenetv2.h5")
valid_loss, valid_acc = model.evaluate(X_valid, y_valid)
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Valid: loss {} acc {} -- Test: loss {} valid {}".format(valid_loss, valid_acc, test_loss, test_acc))

Train on 54000 samples, validate on 6000 samples
Epoch 1/10
54000/54000 [==============================] - 275s 5ms/sample - loss: 2.5760 - acc: 0.9378 - val_loss: 7.3306 - val_acc: 0.1132
Epoch 2/10
54000/54000 [==============================] - 258s 5ms/sample - loss: 0.8636 - acc: 0.9761 - val_loss: 0.9561 - val_acc: 0.8902
Epoch 3/10
54000/54000 [==============================] - 258s 5ms/sample - loss: 0.5217 - acc: 0.9775 - val_loss: 0.9350 - val_acc: 0.7990
Epoch 4/10
54000/54000 [==============================] - 257s 5ms/sample - loss: 0.3839 - acc: 0.9789 - val_loss: 1.5413 - val_acc: 0.7168
Epoch 5/10
54000/54000 [==============================] - 257s 5ms/sample - loss: 0.2934 - acc: 0.9815 - val_loss: 0.6977 - val_acc: 0.8650
Epoch 6/10
54000/54000 [==============================] - 257s 5ms/sample - loss: 0.2506 - acc: 0.9811 - val_loss: 0.3060 - val_acc: 0.9583
Epoch 7/10
54000/54000 [==============================] - 255s 5ms/sample - loss: 0.2223 - acc: 0.9808 - val_lo